In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

### 라벨 비율 확인

In [10]:
import os
import glob

# 라벨이 저장된 폴더 경로 설정
label_folder_path = 'falldown/train'

# 클래스별 개수를 저장할 딕셔너리
class_counts = {}

# 지정된 폴더 내의 모든 txt 파일을 읽음
for label_file in glob.glob(os.path.join(label_folder_path, '*.txt')):
    with open(label_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            class_id = line.split()[0]  # 클래스 ID 추출
            if class_id in class_counts:
                class_counts[class_id] += 1
            else:
                class_counts[class_id] = 1

# 총 개체 수 계산
total_count = sum(class_counts.values())

# 각 클래스의 비율 계산 및 출력
for class_id, count in class_counts.items():
    print(f'Class {class_id}: {count} objects, {count / total_count * 100:.2f}% of total')



Class 0: 27159 objects, 76.41% of total
Class 99: 816 objects, 2.30% of total
Class 2: 6866 objects, 19.32% of total
Class 1: 702 objects, 1.98% of total


### 전처리

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

In [2]:
# 커널 충돌 방지 코드
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
class ComplexPedestrianDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # 이미지 파일을 탐색하고, 각 이미지에 대한 라벨 정보를 로드합니다.
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg'):
                image_path = os.path.join(folder_path, filename)
                label_path = image_path.replace('.jpg', '.txt')  # 라벨 파일 경로

                if os.path.exists(label_path):
                    # 라벨 파일이 존재하면, 라벨 데이터를 파싱합니다.
                    with open(label_path, 'r') as f:
                        labels = [line.strip().split() for line in f.readlines()]
                        labels = [[int(label[0])] + [float(x) for x in label[1:]] for label in labels]
                        self.image_paths.append(image_path)
                        self.labels.append(labels)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        labels = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        # 이미지와 함께 객체별 라벨 정보를 반환합니다.
        return image, labels


# 데이터 증강 작업을 수행함
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

folder_path = 'falldown/train'
dataset = ComplexPedestrianDataset(folder_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)  # 배치 크기는 상황에 따라 조정


In [4]:
import torch
import torch.nn as nn
import torchvision.models as models

class ImageFeatureExtractor(nn.Module):
    def __init__(self):
        super(ImageFeatureExtractor, self).__init__()
        # Pretrained CNN 모델을 사용하여 이미지 특징을 추출합니다.
        # 예를 들어, ResNet50을 사용할 수 있으며, 마지막 분류층을 제외합니다.
        self.feature_extractor = models.resnet50(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(self.feature_extractor.children())[:-2])

    def forward(self, x):
        # CNN을 통과한 이미지 특징 벡터를 반환합니다.
        return self.feature_extractor(x)

class LSTMForObjectDetection(nn.Module):
    def __init__(self, num_classes, hidden_size=256):
        super(LSTMForObjectDetection, self).__init__()
        self.image_feature_extractor = ImageFeatureExtractor()
        self.lstm = nn.LSTM(input_size=2048, hidden_size=hidden_size, batch_first=True)
        # 분류를 위한 출력층: 각 클래스에 대한 예측 확률
        self.fc_class = nn.Linear(hidden_size, num_classes)
        # 위치 예측을 위한 출력층: 중심 좌표(x, y), 너비(w), 높이(h)
        self.fc_bbox = nn.Linear(hidden_size, 4)

    def forward(self, x):
        batch_size = x.size(0)
        # 이미지에서 특징 추출
        features = self.image_feature_extractor(x)  # (batch_size, C, H, W)
        # 특징 벡터를 LSTM의 입력 형식에 맞게 변환
        features = features.permute(0, 2, 3, 1)  # (batch_size, H, W, C)
        features = features.contiguous().view(batch_size, -1, features.size(-1))  # (batch_size, H*W, C)

        # LSTM을 통해 시퀀스 데이터 처리
        lstm_out, _ = self.lstm(features)
        # 최종 시퀀스 출력을 분류 및 위치 예측에 사용
        class_outputs = self.fc_class(lstm_out[:, -1, :])  # 마지막 시퀀스 아이템 사용
        bbox_outputs = self.fc_bbox(lstm_out[:, -1, :])
        
        return class_outputs, bbox_outputs


In [19]:
import torch.optim as optim

# 모델, 데이터셋 클래스, 그리고 다른 필요한 정의들이 이미 완료되었다고 가정합니다.

# 파라미터 설정
num_epochs = 10
learning_rate = 0.001
num_classes = 4  # 0-사람, 1-오인 , 2-실신, 3-폭행, 99-마스킹
hidden_size = 256

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# collate_fn 함수 정의
def collate_fn(batch):
    images, targets = list(zip(*batch))
    # 이미지는 일반적으로 transform을 통해 동일한 크기로 조정되므로 바로 스택을 사용할 수 있습니다.
    images = torch.stack(images, dim=0).to(device)

    # 타겟 처리
    # 가장 긴 타겟을 찾습니다.
    max_len = max(len(t) for t in targets)
    # 모든 타겟이 동일한 길이를 갖도록 패딩을 추가합니다.
    padded_targets = []
    for t in targets:
        padded = t + [[-1, -1, -1, -1, -1]] * (max_len - len(t))  # 예시 패딩 값으로는 -1을 사용
        padded_targets.append(padded)

    targets_padded = torch.tensor(padded_targets, dtype=torch.float).to(device) 
    return images, targets_padded

# 데이터셋 및 데이터 로더 초기화
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
train_dataset = ComplexPedestrianDataset(folder_path='falldown/train', transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

# 모델 초기화
model = LSTMForObjectDetection(num_classes=num_classes, hidden_size=hidden_size).to(device)

# 손실 함수와 옵티마이저
criterion_class = torch.nn.CrossEntropyLoss()  # 분류 손실
criterion_bbox = torch.nn.MSELoss()  # 회귀 손실(바운딩 박스)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [18]:
train_dataset

In [15]:
train_loader

In [ ]:

# 학습 루프 수정
for epoch in range(num_epochs):
    for images, targets in train_loader:
        optimizer.zero_grad()

        # 모델 예측
        class_outputs, bbox_outputs = model(images)

        # targets 내용을 확인하고 적절히 처리
        class_labels = []
        bbox_labels = []
        for batch_item  in targets:
            # 여기서 각 target은 [class_label, x_center, y_center, width, height]의 형태를 가짐
            for obj in batch_item:
                class_label, bbox = obj[0], obj[1:]
                class_labels.append(class_label)
                bbox_labels.extend(bbox) 

        # 이제 class_labels와 bbox_labels를 PyTorch 텐서로 변환할 수 있습니다.
        class_labels_tensor = torch.tensor(class_labels, dtype=torch.long).to(images.device)
        bbox_labels_tensor = torch.tensor(bbox_labels, dtype=torch.float).to(images.device).view(-1, 4)

        # print(class_labels_tensor.size())
        # print(bbox_labels_tensor.size())

        # 손실 계산
        loss_class = criterion_class(class_outputs, class_labels_tensor)
        loss_bbox = criterion_bbox(bbox_outputs, bbox_labels_tensor)
        total_loss = loss_class + loss_bbox


        # 역전파 및 옵티마이저 스텝
        total_loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss.item():.4f}')